In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import torch as torch
import nltk
from datasets import load_dataset
import torch 


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cpu



In [3]:
dataset = load_dataset('csv', data_files = 'spotify_millsongdata.csv')

Found cached dataset csv (C:/Users/Heshy/.cache/huggingface/datasets/csv/default-004d8cf9f45e9b24/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
from transformers import GPT2Tokenizer, GPT2Model
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from transformers import GPT2TokenizerFast


model = GPT2Model.from_pretrained('gpt2-medium')
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
if tokenizer.pad_token is None:
    tokenizer.pad_token = '[PAD]'
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

Using pad_token, but it is not set yet.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50258. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [5]:
encoded_input = tokenizer("melodie gen abba", return_tensors="pt")
encoded_input


{'input_ids': tensor([[17694,   375,   494,  2429,   450,  7012]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [6]:
MAX_LENGTH = 256
BATCH_SIZE = 6
tokenized_data = tokenizer(
    dataset['train']['text'],
    truncation = True,
    max_length = MAX_LENGTH,
    return_overflowing_tokens = True,
    return_length = True
)
tokenized_data = {"input_ids": tokenized_data["input_ids"]}


In [7]:
def preprocess(example):
    try:
        outputs = tokenizer(
        example["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        return_overflowing_tokens=True,
        return_length=True,
        padding="max_length",  # Aceasta va utiliza pad_token implicit
        return_tensors="pt"
    )

        
        input_batch = outputs["input_ids"].tolist()

        if len(input_batch) < BATCH_SIZE:
            input_batch += [input_batch[0]] * (BATCH_SIZE - len(input_batch))
    except Exception as e:
        print(f"An error occurred: {e}")
        input_batch = []

    return {"input_ids": input_batch}


tokenized_dataset = dataset.map(preprocess, remove_columns = dataset['train'].column_names)


Loading cached processed dataset at C:\Users\Heshy\.cache\huggingface\datasets\csv\default-004d8cf9f45e9b24\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b312978bb9f2b727.arrow


In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 57650
    })
})

In [9]:
def filter_none(example):
    if len(example['input_ids']) >= 1:
        return example
final_data = tokenized_dataset.filter(filter_none)
final_data



Loading cached processed dataset at C:\Users\Heshy\.cache\huggingface\datasets\csv\default-004d8cf9f45e9b24\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-65a72645aea9cc86.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 57650
    })
})

In [10]:
tokenizer.pad_token = tokenizer.eos_token
data_col = DataCollatorForLanguageModeling(tokenizer, mlm = False, return_tensors = "tf") # mlm = masked language modeling

tf_train_dataset = final_data["train"].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'labels'],
    shuffle = True,
    batch_size = 1,
    collate_fn = data_col
)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [11]:
import tensorflow as tf

def swap_position(example):
    return {
        'input_ids': example['input_ids'],
        'attention_mask': tf.ones([1, BATCH_SIZE, MAX_LENGTH]),
        'labels': example['labels']
    }
train_data = tf_train_dataset.map(swap_position)
unbatched_train_data = train_data.unbatch()


In [12]:
from transformers import TFGPT2LMHeadModel
model = TFGPT2LMHeadModel.from_pretrained('gpt2-medium')
model.summary()

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  354823168 
 er)                                                             
                                                                 
Total params: 354823168 (1.32 GB)
Trainable params: 354823168 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
from transformers import create_optimizer
num_train_steps = len(unbatched_train_data)
optimizer, schedule = create_optimizer(
    init_lr = 5e-5,
    num_warmup_steps = 1_000,
    num_train_steps = num_train_steps
)

model.compile(
    optimizer = optimizer
)


In [16]:
train_history = model.fit(unbatched_train_data, epochs = 5)


Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'tfgpt2lm_head_model/transformer/assert_less/Assert/Assert' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\Heshy\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Heshy\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\Heshy\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\Heshy\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Heshy\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Users\Heshy\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once
    